In [ ]:
import numpy
import pandas

import seaborn as sns

from matplotlib import pyplot


sns.set_theme(font="IPAexGothic")
pyplot.rcParams["figure.figsize"] = (16, 8)

In [ ]:
from datetime import datetime

In [ ]:
g_start_date = datetime.now()
g_start_date.strftime("%Y/%m/%d %T")

In [ ]:
url = "https://www.mof.go.jp/jgbs/reference/interest_rate/data/jgbcm_all.csv"
_df = pandas.read_csv(url, header=1, encoding="shift-jis")
_df.tail()

In [ ]:
# NOTE: created by ChatGPT GPT-4
def convert_wareki_to_seireki(wareki_date):
    """
    和暦を西暦に変換する関数。
    """
    era = wareki_date[0]
    year, month, day = map(int, wareki_date[1:].split("."))

    if era == "S":  # 昭和
        seireki_year = 1925 + year
    elif era == "H":  # 平成
        seireki_year = 1988 + year
    elif era == "R":  # 令和
        seireki_year = 2018 + year
    else:
        raise ValueError(f"Unknown era: {era}")

    return f"{seireki_year}-{month:02}-{day:02}"

In [ ]:
_df["ds"] = pandas.to_datetime(_df["基準日"].apply(convert_wareki_to_seireki))
_df[["基準日", "ds"]]

In [ ]:
_df.replace("-", float("nan"), inplace=True)

In [ ]:
_df.head()

In [ ]:
rate_cols = [col for col in _df.columns if col not in ["基準日", "ds"]]
rate_cols = ["5年", "10年", "20年"]
rate_cols

In [ ]:
_df[rate_cols] = _df[rate_cols].astype(float)
_df.head(3)

In [ ]:
for col in rate_cols:
    pyplot.plot(_df["ds"], _df[col], label=col)

pyplot.title("日本国債金利の変動")
pyplot.xlabel("ds")
pyplot.ylabel("金利 (%)")
pyplot.legend()
pyplot.grid(True)
pyplot.tight_layout()
pyplot.show()

In [ ]:
df = _df.copy()
df.index = pandas.to_datetime(df.ds)

In [ ]:
df_weekly = df[rate_cols].resample("W-MON").mean()
df_weekly.plot()
pyplot.show()

In [ ]:
df_weekly.tail(3)

In [ ]:
df_monthly = df[rate_cols].resample("MS").mean()
# df_monthly["2000-01-01":].plot()
df_monthly.plot()
pyplot.show()

In [ ]:
df_monthly = df[rate_cols].resample("MS").mean()
df_monthly.head(3)

In [ ]:
df_monthly.tail(3)

In [ ]:
len(df_monthly)

In [ ]:
import optuna

from prophet import Prophet

In [ ]:
from protuna import (
    ModelBuilder,
    Evaluator,
    BestEstimator,
    ProphetModelAnalyser,
    optuna_visualization,
)

# シミュレーション

In [ ]:
g_rate_types = rate_cols

## 実験パラメータ

In [ ]:
start_date = "2000-01-01"
period_type = "monthly"
# start_date = "2015-01-01"
# period_type = "weekly"
n_trials = 6

if period_type == "weekly":
    df0 = df_weekly[start_date:].reset_index()
    freq = "W-MON"
    freq_cv = "4W-MON"
    # NOTE: for long
    n_horizon = 7 * 4 * 6  # days: about 6 months
    horizon_scaler = 5
    n_predicts = 4 * 6  # number of freqs: 6か月分
    # NOTE: for short
    # n_horizon = 7 * 4  # days: about 1 month
    # horizon_scaler = 2
    # n_predicts = 4 * 1  # number of freqs: 1か月分
else:
    df0 = df_monthly[start_date:].reset_index()
    freq = "MS"
    freq_cv = "3MS"  # simulation by quarter
    n_horizon = 365.25 * 3  # days: about 3 year
    horizon_scaler = 5
    n_predicts = 12 * 5  # number of freqs: 5年分

# NOTE: ざっくり、データ数を日数換算(ここでは、単純に x 30)した時に、
#       n_horizon * (1 + horizon_scaler)が超えないこと
assert n_horizon * (1 + horizon_scaler) <= len(df0) * 30

In [ ]:
len(df0) * 30

In [ ]:
df0.head(3)

In [ ]:
df0.tail(3)

In [ ]:
n_horizon

In [ ]:
# NOTE:
#   - weekly: around 90 x n (rate_types) mins
#   - monthly: around 5 x n (rate_types) mins

bests = {}

for rate_type in g_rate_types:
    # setup
    df = df0.rename({"年月": "ds", rate_type: "y"}, axis=1)[["ds", "y"]]
    df["fake"] = list(range(len(df)))
    cap = df.y.max() + 3 * df.y.std()
    df["cap"] = cap
    evl = Evaluator(
        df=df, n_horizon=n_horizon, freq=freq_cv, horizon_scaler=horizon_scaler
    )

    # optimize hyper params
    study: optuna.Study = optuna.create_study(direction="minimize")
    study.optimize(evl.objective_value, n_trials=n_trials)

    # rerun cv
    mb: ModelBuilder = ModelBuilder(df=df)
    best_params = study.best_params.copy()
    cap_scaler = best_params.pop("cap_scaler")
    model: Prophet = mb.build_prophet_model(**best_params)
    df["cap"] = df.y.max() + cap_scaler * df.y.std()
    model.fit(df)
    df_cv, df_pm = evl.run_cross_validation(model=model)

    # predict
    future = model.make_future_dataframe(periods=n_predicts, freq=freq)
    future["cap"] = cap
    forecast = model.predict(future)

    # store estimator
    bst = BestEstimator(
        df=df,
        model=model,
        evaluator=evl,
        study=study,
        df_cv=df_cv,
        df_pm=df_pm,
        future=future,
        forecast=forecast,
    )
    bests[rate_type] = bst
    break  # for debugging

In [ ]:
pmz = ProphetModelAnalyser(model=model, df=df)
beta_juglar = pmz.pickup_beta(component="juglar_10")
beta_juglar

In [ ]:
study.best_params

In [ ]:
bst.df_pm.columns

# 可視化

In [ ]:
from prophet.plot import plot_plotly, plot_components_plotly


n_years = 1
n_predicts = 12 * n_years


for rate_type in bests.keys():
    bst: BestEstimator = bests[rate_type]

    # predict
    model: Prophet = bst.model
    future = model.make_future_dataframe(periods=n_predicts, freq=freq)

    # setup "cap"
    best_params = bst.study.best_params.copy()
    cap_scaler: float = best_params.pop("cap_scaler")
    cap = bst.df.y.max() + cap_scaler * bst.df.y.std()
    future["cap"] = cap

    forecast = model.predict(future)

    # plot prediction
    fig: pyplot.Figure = model.plot(forecast, figsize=(20, 10))
    fig.suptitle(f"実績と予測: {rate_type}", x=0.5, y=1.02, size=16)
    pyplot.axvline(bst.df.ds.iloc[-1], color="b", linestyle="--", lw=1)
    fig.show()

    # plot components
    fig = model.plot_components(forecast, figsize=(20, 20))
    fig.suptitle(f"コンポーネント: {rate_type}", x=0.5, y=1.02, size=16)
    fig.show()

    # plot errors
    bst.df_pm.assign(horizon=bst.df_pm.horizon.dt.days).set_index("horizon")[
        ["rmse", "mae"]
    ].plot()
    pyplot.title(f"誤差: {rate_type}")
    pyplot.show()
    metrics_key = []
    if "mape" in bst.df_pm:
        metrics_key += ["mape"]
    metrics_key += ["smape", "mdape"]
    bst.df_pm.assign(horizon=bst.df_pm.horizon.dt.days).set_index("horizon")[
        metrics_key
    ].plot()
    pyplot.title(f"誤差率: {rate_type}")
    pyplot.show()

In [ ]:
bst.df_pm

In [ ]:
bst.study.best_params

In [ ]:
pandas.DataFrame({k: [v] for k, v in bst.study.best_params.items()}).T  # .plot()

In [ ]:
from IPython.display import display, Markdown


for rt in bests.keys():
    bst: BestEstimator = bests[rt]
    display(
        Markdown(
            f"""---
# 国債金利：{rt} 
"""
        )
    )
    display(
        pandas.DataFrame(
            {k: [v] for k, v in bst.study.best_params.items()}, index=["value"]
        ).T
    )
    optuna_visualization(bst.study)

In [ ]:
rate_type = rate_cols[0]  # 最初の金利
bst: BestEstimator = bests[rate_type]
rate_type

In [ ]:
import plotly.graph_objs as go

In [ ]:
# changepoint data
changepoints_threshold = 0.01
signif_changepoints = bst.model.changepoints[
    numpy.abs(numpy.nanmean(bst.model.params["delta"], axis=0))
    >= changepoints_threshold
]
df_cp = signif_changepoints.reset_index(drop=True)
df_cp

In [ ]:
fig: go.Figure = plot_plotly(bst.model, bst.forecast, trend=True, changepoints=True)
fig.update_layout(title=f"予測 : {rate_type}")

In [ ]:
plot_components_plotly(model, forecast).update_layout(title=f"コンポーネント : {rate_type}")

In [ ]:
bst.study.best_params

In [ ]:
g_end_date = datetime.now()
g_end_date.strftime("%Y/%m/%d %T")

In [ ]:
bst.model.n_changepoints

In [ ]:
freq

In [ ]:
base_dates = pandas.date_range(
    start=bst.df.ds.min(), end=bst.future.ds.max(), freq=freq, name="ds"
)
base_dates = base_dates.to_frame().reset_index(drop=True)
base_dates.head(3)
base_dates.tail(3)

In [ ]:
bst.study.best_params

In [ ]:
df_forecast = bst.forecast.copy()
df_forecast.index = pandas.to_datetime(df_forecast.ds)
seasonality_cols = ["yearly", "triennial", "kitchen", "quinquennial", "juglar_10"]

df_forecast["seasonality"] = (
    df_forecast[seasonality_cols]
    .sum(axis=1)
    .rename("seasonality")
    .to_frame()
    .set_index(df_forecast.ds)
)
df_forecast["y"] = bst.df.y

pyplot.clf()
# ax = df_forecast[["y"]].plot(color="r")
# ax = df_forecast[["trend"]].plot(kind="line", ax=ax, color="green")
# ax = df_forecast[["yhat"]].plot(kind="line", ax=ax, color="b")
# ax = df_forecast[["seasonality"]].plot(kind="line", ax=ax, color="#FFB83F")
ax = df_forecast[["yhat", "y", "seasonality", "trend"]].plot()
ax.fill_between(
    df_forecast.index,
    df_forecast.yhat_lower,
    df_forecast.yhat_upper,
    color="#0072B2",
    alpha=0.2,
    label="Uncertainty interval",
)
ax.axvline(bst.df.ds.iloc[-1], color="b", linestyle="--", lw=2)
pyplot.show()

In [ ]:
use_horizon_actual = True

for rt in bests.keys():
    bst: BestEstimator = bests[rt]
    bst.df.index = bst.df.ds

    display(
        Markdown(
            f"""---
# 国債金利 : {rt}
"""
        )
    )

    seasonality_cols = ["yearly", "triennial", "kitchen", "quinquennial", "juglar_10"]
    n_years = 5
    date_start = bst.df.ds.iloc[-1] - pandas.Timedelta(n_years * 365.25, "days")
    date_end = bst.df.ds.iloc[-1]
    cutoffs = pandas.date_range(start=date_start, end=date_end, freq=freq_cv)

    for ctf in cutoffs:
        _df = bst.df.copy()
        df = _df[["y", "ds"]].set_index(pandas.to_datetime(_df.ds)).loc[:ctf]
        mb: ModelBuilder = ModelBuilder(df=df)
        best_params = bst.study.best_params.copy()
        cap_scaler = best_params.pop("cap_scaler")
        model: Prophet = mb.build_prophet_model(**best_params)
        df["cap"] = cap = df.y.max() + cap_scaler * df.y.std()
        model.fit(df)
        future = bst.model.make_future_dataframe(periods=n_predicts, freq=freq)

        # setup "cap"
        best_params = bst.study.best_params.copy()
        future["cap"] = cap
        forecast = bst.model.predict(future)
        forecast.index = pandas.to_datetime(forecast.ds)
        forecast["seasonality"] = (
            forecast[seasonality_cols]
            .sum(axis=1)
            .rename("seasonality")
            .to_frame()
            .set_index(pandas.to_datetime(forecast.ds))
        )

        # df_plotter = pandas.merge(base_dates, grp.reset_index(), on="ds", how="left")
        df_plotter = pandas.merge(
            base_dates, forecast.reset_index(drop=True), on="ds", how="left"
        )
        df_plotter.index = pandas.to_datetime(df_plotter["ds"])
        df_plotter["y"] = bst.df.y.loc[:ctf]
        df_plotter["trend"] = forecast["trend"].loc[:ctf]
        df_plotter["seasonality"] = forecast["seasonality"].loc[:ctf]
        fig = pyplot.figure(facecolor="lightgrey", figsize=(20, 10))
        ax = fig.add_subplot(111)
        df_plotter[["yhat", "y", "seasonality", "trend"]].plot(ax=ax)
        ax.fill_between(
            df_plotter.index,
            df_plotter.yhat_lower,
            df_plotter.yhat_upper,
            color="#0072B2",
            alpha=0.2,
            label="Uncertainty interval",
        )
        ax.axvline(ctf, color="b", linestyle="--", lw=2)
        ax.set_ylim(-1.5, 3)

        pyplot.title(f"cutoff: {ctf}", fontsize=20)
        pyplot.show()

# 実験管理

In [ ]:
import mlflow

experiment_name = "jgbcm_interest_rate"

tracking_uri = f"sqlite:///../data/experiment.db"
mlflow.set_tracking_uri(tracking_uri)

experiment = mlflow.get_experiment_by_name(name=experiment_name)
if experiment is None:
    experiment_id = mlflow.create_experiment(name=experiment_name)
else:
    experiment_id = experiment.experiment_id

In [ ]:
# NOTE: Enable to infer input signature of `_ds`, for MLflow
# str_cols = ["基準日", "ds"]
str_cols = ["ds"]
value_cols = [col for col in _df.columns if col not in str_cols]
_df = _df.assign(**{col: _df[col].astype(float) for col in value_cols}).assign(
    **{col: _df[col].astype(str) for col in str_cols}
)
_df[value_cols].head(3)

In [ ]:
import yaml
from mlflow.models import infer_signature
from mlflow.data.pandas_dataset import PandasDataset


desc = "Predict the interest rate for jgbcm"
with mlflow.start_run(
    experiment_id=experiment_id, run_name="trial", description=desc
) as mlf:
    ds: PandasDataset = mlflow.data.from_pandas(_df, source=url)
    mlflow.log_input(ds, context="simulation")

    mlflow.set_tag("start", g_start_date)
    mlflow.set_tag("end", g_end_date)
    duration = round((g_end_date - g_start_date).seconds / 60, 0)
    mlflow.set_tag("duration", f"{duration} mins")

    mlflow.log_param("start_date", f"{start_date}")
    mlflow.log_param("period_type", f"{period_type}")
    mlflow.log_param("freq", f"{freq}")
    mlflow.log_param("freq_cv", f"{freq_cv}")
    mlflow.log_param("n_horizon", f"{n_horizon}")
    mlflow.log_param("horizon_scaler", f"{horizon_scaler}")
    mlflow.log_param("n_predicts", f"{n_predicts}")
    mlflow.log_param("n_trials", f"{n_trials}")

    for rt in bests.keys():
        bst: BestEstimator = bests[rt]

        # params
        best_params: dict = {f"{rt}/{k}": v for k, v in bst.study.best_params.items()}
        mlflow.log_params(best_params)
        mlflow.log_param(f"{rt}/n_changepoints", f"{bst.model.n_changepoints}")
        mlflow.log_param(f"{rt}/changepoint_range", f"{bst.model.changepoint_range}")
        params_file = "params.yaml"
        with open(params_file, "w") as fw:
            yaml.dump(bst.study.best_params, fw)
        mlflow.log_artifact(params_file, artifact_path=f"{rt}")

        # metrics
        metrics: dict = bst.df_pm.mean().drop(["horizon", "coverage"]).to_dict()
        metrics = {f"{rt}/{k}": v for k, v in metrics.items()}
        mlflow.log_metrics(metrics=metrics)

        # model
        signature = infer_signature(bst.df, bst.forecast)
        mlflow.prophet.log_model(bst.model, artifact_path=f"{rt}", signature=signature)
        # mlflow.evaluate()

In [ ]:
%%bash
rm -f params.yaml